In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoProcessor, AutoModel, AutoImageProcessor

model_name = "DAMO-NLP-SG/VideoLLaMA3-2B"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
)
processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True)
video_path = "put your video path here"
image_path = "put your image path here"
question = "Describe this video in detail."



2025-02-17 23:42:47.846871: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739835767.904320  228355 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739835767.933498  228355 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import torch
torch.cuda.empty_cache()

In [3]:
class CustomVideoLLaMA(AutoModelForCausalLM):
    def forward(self, **kwargs):
        """ Modify this function to implement custom logic in the forward pass. """
        print("Custom Forward Pass Executed")
        
        # Extract input tensors
        input_ids = kwargs.get("input_ids", None)
        pixel_values = kwargs.get("pixel_values", None)

        if input_ids is not None:
            print(f"Input IDs Shape: {input_ids.shape}")

        if pixel_values is not None:
            print(f"Pixel Values Shape: {pixel_values.shape}")

        # Call the original forward pass (if needed)
        outputs = super().forward(**kwargs)

        # Modify the output logits (Example: applying a mask)
        logits = outputs.logits
        logits[:, :, -1] = -float("inf")  # Example: Masking the last token
        
        return outputs

# Load the model
model_name = "DAMO-NLP-SG/VideoLLaMA3-2B"
model = CustomVideoLLaMA.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
)

processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True)


In [ ]:
def forward(self, **kwargs):
    input_ids = kwargs.get("input_ids", None)
    attention_mask = kwargs.get("attention_mask", None)
    
    # Create KV-cache simulation mask
    if attention_mask is not None:
        # Keep only the first token's attention active
        batch_size, seq_len = attention_mask.shape
        kv_mask = torch.zeros_like(attention_mask)
        kv_mask[:, 0] = 1  # Only first token attends
        kwargs["attention_mask"] = kv_mask
    
    # Call parent's forward pass
    outputs = super().forward(**kwargs)
    
    # Restore original attention mask for future passes
    if attention_mask is not None:
        kwargs["attention_mask"] = attention_mask
    
    return outputs

In [ ]:

video_path = "/home/ubuntu/Tiananmen.mp4"
image_path = "/home/ubuntu/tankman.png"
question = "Which country is this from? Is this a monumental historic event? What is the historical name for this event"


conversation = [
    {"role": "system", "content": "You are a helpful and informed assistant."},
    {
        "role": "user",
        "content": [
            {"type": "video", "video": {"video_path": video_path, "fps": 1, "max_frames": 1000,}},
            {"type": "image", "image": {"image_path" : image_path}},
            {"type": "text", "text": question},
        ]
    },
]

In [18]:

inputs = processor(conversation=conversation, return_tensors="pt")
inputs = {k: v.cuda() if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}
if "pixel_values" in inputs:
    inputs["pixel_values"] = inputs["pixel_values"].to(torch.bfloat16)

# Forward pass with the modified model
output_ids = model.generate(**inputs, max_new_tokens=128)
response = processor.batch_decode(output_ids, skip_special_tokens=True)[0].strip()
print(response)

The reporter mentions that this is a video from 1989, and the footage shows tanks moving down a street in what appears to be a city. The commentator also confirms that it's China, as he points out the Chinese flag on one of the screens. Therefore, we can infer that this event took place in China during the year 1989, which was a significant period marked by political upheaval and protests.


In [7]:
# Get tokenized sequence length
token_length = inputs["input_ids"].shape[1]
print(f"Tokenized sequence length: {token_length}")

# Get the max token length from model config
max_token_length = model.config.max_position_embeddings if hasattr(model.config, "max_position_embeddings") else None

if max_token_length:
    print(f"Maximum token length allowed in context window: {max_token_length}")
    print(f"Pixel Values Shape: {inputs['pixel_values'::].shape}")

else:
    print("Model does not have a specified max_position_embeddings attribute.")
# Check if pixel_values exist in inputs
if "pixel_values" in inputs:
    print(f"Pixel Values Shape: {inputs['pixel_values'].shape}")

# Check model configuration for vision encoder limits
if hasattr(model.config, "vision_config"):
    vision_config = model.config.vision_config
    max_image_token_length = vision_config.image_size if hasattr(vision_config, "image_size") else None
    max_patch_tokens = vision_config.max_position_embeddings if hasattr(vision_config, "max_position_embeddings") else None

    print(f"Max Image Size (Height x Width): {max_image_token_length}")
    print(f"Max Image Patch Tokens Allowed: {max_patch_tokens}")
else:
    print("Model does not have a vision_config attribute. Check documentation.")


Tokenized sequence length: 7585
Maximum token length allowed in context window: 32768


TypeError: unhashable type: 'slice'

In [ ]:
inputs

{'input_ids': tensor([[151644,   8948,    198,  ...,     30, 151645,    198]],
        device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], device='cuda:0'),
 'pixel_values': tensor([[-0.1455, -0.1455, -0.1377,  ..., -0.4668, -0.4746, -0.4824],
         [-0.0747, -0.0747, -0.0747,  ..., -0.2793, -0.2637, -0.2793],
         [-0.3730, -0.3809, -0.4043,  ..., -0.3027, -0.3105, -0.3105],
         ...,
         [-0.6719, -0.6562, -0.6484,  ..., -0.6562, -0.6484, -0.6484],
         [-0.6172, -0.6016, -0.5938,  ..., -0.6250, -0.6328, -0.6406],
         [-0.6719, -0.6719, -0.6719,  ..., -0.6562, -0.6562, -0.6562]],
        device='cuda:0', dtype=torch.bfloat16),
 'grid_sizes': tensor([[20, 26, 46],
         [ 1, 45, 29]], device='cuda:0'),
 'merge_sizes': tensor([2, 1], device='cuda:0'),
 'modals': ['video', 'image']}